In [2]:
rng_key = jax.random.PRNGKey(0)
A = jax.random.normal(rng_key, shape=(3, 2))

B = jnp.einsum('ij,jk->ijk', A, jnp.eye(2))

print(A)

NameError: name 'jax' is not defined

In [3]:
print(B)

NameError: name 'B' is not defined

In [4]:
B[0, :, :]

NameError: name 'B' is not defined

In [5]:
rng_key = jax.random.PRNGKey(0)
A = jax.random.normal(rng_key, shape=(3, 3, 2, 2))
print(A)

NameError: name 'jax' is not defined

In [84]:
A[:, :, 0, 0]

Array([[-0.19596423, -0.847186  , -0.88885146],
       [-0.44171792,  1.0892068 , -1.222362  ],
       [-0.33807516,  0.2918976 ,  0.11422886]], dtype=float32)

In [85]:
jnp.diagonal(A, axis1=0, axis2=1).shape

(2, 2, 3)

In [86]:
jnp.diagonal(A, axis1=0, axis2=1).transpose((2, 0, 1))

Array([[[-0.19596423,  0.254582  ],
        [ 0.573146  ,  0.44064867]],

       [[ 1.0892068 , -0.8457174 ],
        [ 1.490981  ,  0.07877276]],

       [[ 0.11422886,  0.25447085],
        [-1.3060236 , -2.473075  ]]], dtype=float32)

In [87]:
jnp.diagonal(A, axis1=0, axis2=1).transpose((2, 0, 1))[:, 0, 0]

Array([-0.19596423,  1.0892068 ,  0.11422886], dtype=float32)

(2, 4)

In [24]:
A + B

Array([[[2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]],

       [[2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]],

       [[2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]]], dtype=float32)

In [28]:
jnp.linalg.det(B).shape

(3,)

In [32]:
import jax.numpy as jnp
import jax
import time
# Example usage:
N1 = 3
N2 = 4
N3 = 2
D = 6

rng_key = jax.random.PRNGKey(0)
x = jax.random.normal(rng_key, shape=(N1, N2, D))

rng_key, _ = jax.random.split(rng_key)
mean = jax.random.normal(rng_key, shape=(N3, D))

rng_key, _ = jax.random.split(rng_key)
covariance = jax.random.uniform(rng_key, shape=(N3, D))
covariance = jnp.stack([jnp.diag(c) for c in covariance.reshape(-1, D)], axis=0).reshape(N3, D, D)


x_expanded = jnp.expand_dims(x, 2)
mean_expanded = jnp.expand_dims(mean, (0, 1))
# covariance_expanded = jnp.expand_dims(covariance, 0)


t0 = time.time()
diff = x_expanded - mean_expanded
precision_matrix = jnp.linalg.inv(covariance)
exponent = -0.5 * jnp.einsum('nijk,jkl,nijl->nij', diff, precision_matrix, diff)
normalization = -0.5 * (D * jnp.log(2 * jnp.pi) - 0.5 * jnp.log(jnp.linalg.det(covariance)))
log_likelihood = exponent + normalization
print(time.time() - t0)
# print(log_likelihood, log_likelihood.shape)
# print(exponent, exponent.shape)

t0 = time.time()
diff_1 = diff.reshape([N1, N2, N3, D, 1])
precision_matrix_ = precision_matrix.reshape([1, 1, N3, D, D])
E = (diff_1 * precision_matrix_).sum(3)
# E_ = jnp.expand_dims(E, 2)
normalization = -0.5 * (D * jnp.log(2 * jnp.pi) - 0.5 * jnp.log(jnp.linalg.det(covariance)))
exponent = -0.5 * (E * diff).sum(-1)
log_likelihood = exponent + normalization
print(time.time() - t0)
# print(exponent, exponent.shape)


0.001641988754272461
0.0017199516296386719


In [24]:
diff.shape

(3, 4, 2, 6)

In [7]:
precision_matrix.shape

(2, 6, 6)

In [21]:
log_likelihood = jnp.zeros([N1, N2, N3])

for n1 in range(N1):
    for n2 in range(N2):
        for n3 in range(N3):
            diff = x[n1, n2, :] - mean[n3, :]
            precision_matrix = jnp.linalg.inv(covariance[n3, :, :])
            exponent = -0.5 * jnp.dot(diff.T, jnp.dot(precision_matrix, diff))
            normalization = -0.5 * (D * jnp.log(2 * jnp.pi) + 0.5 * jnp.log(jnp.linalg.det(precision_matrix)))
            log_likelihood = log_likelihood.at[n1, n2, n3].set(exponent + normalization)
            
print(log_likelihood)


[[[ -44.087975  -29.820477]
  [ -16.644535  -11.319531]
  [ -47.74982   -22.784111]
  [ -17.861134  -19.58884 ]]

 [[ -54.307503  -15.372691]
  [-222.3221    -15.377424]
  [ -37.44995   -13.018433]
  [ -20.321373  -38.71425 ]]

 [[ -61.859123  -16.834034]
  [ -34.70679   -15.578803]
  [ -48.57657   -33.38154 ]
  [ -78.95834   -14.548405]]]


In [113]:
log_likelihood.shape

(3, 4, 2)

In [57]:
normalization.shape

(4, 2)

In [54]:
covariance.shape

(4, 2, 3, 3)

In [33]:
import jax
import jax.numpy as jnp
import timeit

# Define two random matrices for the comparison
A = jax.random.normal(jax.random.PRNGKey(0), (1000, 1000))
B = jax.random.normal(jax.random.PRNGKey(1), (1000, 1000))

def einsum_multiply():
    global A, B
    C = jnp.einsum('ij,jk->ik', A, B)

def dot_multiply():
    global A, B
    C = jnp.dot(A, B)

# Compile the functions using jax.jit
einsum_multiply_compiled = jax.jit(einsum_multiply)
dot_multiply_compiled = jax.jit(dot_multiply)

# Run the compiled functions once to compile the JAX computations
_ = einsum_multiply_compiled()
_ = dot_multiply_compiled()

# Use timeit to measure the time it takes to execute the compiled functions
einsum_time = timeit.timeit(einsum_multiply_compiled, number=100)
dot_time = timeit.timeit(dot_multiply_compiled, number=100)

print(f"Time for einsum: {einsum_time:.6f} seconds")
print(f"Time for dot: {dot_time:.6f} seconds")


Time for einsum: 0.006879 seconds
Time for dot: 0.005470 seconds


In [87]:
seed = 1
rng_key = jax.random.PRNGKey(seed)

X = jax.random.normal(rng_key, shape=(10, 2))
poly = 3
X_poly = jnp.ones_like(X)
for i in range(1, poly + 1):
    X_poly = jnp.concatenate((X_poly, X ** i), axis=1)

In [88]:

powers = jnp.arange(0, poly + 1)
X_ = X[:, :, None]
X_power = X_ ** powers

In [89]:
X

Array([[-0.9859176 , -1.0066605 ],
       [-0.2213709 , -1.3452404 ],
       [ 0.74275523, -2.1406245 ],
       [-0.6595632 , -0.06608305],
       [-0.21085918, -0.35348117],
       [-1.4126911 , -1.9433796 ],
       [ 1.6297984 ,  0.24093218],
       [-1.7616359 , -0.7132233 ],
       [ 0.86033255, -0.19197786],
       [ 0.5360398 , -0.5192681 ]], dtype=float32)

In [90]:
X_poly

Array([[ 1.0000000e+00,  1.0000000e+00, -9.8591763e-01, -1.0066605e+00,
         9.7203356e-01,  1.0133653e+00, -9.5834500e-01, -1.0201148e+00],
       [ 1.0000000e+00,  1.0000000e+00, -2.2137091e-01, -1.3452404e+00,
         4.9005076e-02,  1.8096716e+00, -1.0848298e-02, -2.4344432e+00],
       [ 1.0000000e+00,  1.0000000e+00,  7.4275523e-01, -2.1406245e+00,
         5.5168533e-01,  4.5822735e+00,  4.0976718e-01, -9.8089266e+00],
       [ 1.0000000e+00,  1.0000000e+00, -6.5956318e-01, -6.6083051e-02,
         4.3502361e-01,  4.3669697e-03, -2.8692555e-01, -2.8858267e-04],
       [ 1.0000000e+00,  1.0000000e+00, -2.1085918e-01, -3.5348117e-01,
         4.4461593e-02,  1.2494894e-01, -9.3751354e-03, -4.4167098e-02],
       [ 1.0000000e+00,  1.0000000e+00, -1.4126911e+00, -1.9433796e+00,
         1.9956962e+00,  3.7767243e+00, -2.8193023e+00, -7.3396091e+00],
       [ 1.0000000e+00,  1.0000000e+00,  1.6297984e+00,  2.4093218e-01,
         2.6562428e+00,  5.8048315e-02,  4.3291402e+00,  1

In [91]:
X_power.shape

(10, 2, 4)

In [92]:
X_power.reshape([X_power.shape[0], 8])

Array([[ 1.0000000e+00, -9.8591763e-01,  9.7203356e-01, -9.5834506e-01,
         1.0000000e+00, -1.0066605e+00,  1.0133653e+00, -1.0201148e+00],
       [ 1.0000000e+00, -2.2137091e-01,  4.9005076e-02, -1.0848299e-02,
         1.0000000e+00, -1.3452404e+00,  1.8096716e+00, -2.4344432e+00],
       [ 1.0000000e+00,  7.4275523e-01,  5.5168533e-01,  4.0976718e-01,
         1.0000000e+00, -2.1406245e+00,  4.5822735e+00, -9.8089266e+00],
       [ 1.0000000e+00, -6.5956318e-01,  4.3502361e-01, -2.8692555e-01,
         1.0000000e+00, -6.6083051e-02,  4.3669697e-03, -2.8858267e-04],
       [ 1.0000000e+00, -2.1085918e-01,  4.4461593e-02, -9.3751354e-03,
         1.0000000e+00, -3.5348117e-01,  1.2494894e-01, -4.4167098e-02],
       [ 1.0000000e+00, -1.4126911e+00,  1.9956962e+00, -2.8193023e+00,
         1.0000000e+00, -1.9433796e+00,  3.7767243e+00, -7.3396091e+00],
       [ 1.0000000e+00,  1.6297984e+00,  2.6562428e+00,  4.3291402e+00,
         1.0000000e+00,  2.4093218e-01,  5.8048315e-02,  1

In [94]:
import time
import numpy as np
import jax.numpy as jnp

def measure_time(func, *args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    return result, end_time - start_time

def compare_speeds(n):
    np.random.seed(0)
    A = np.random.rand(n, n)
    B = np.random.rand(n, n)

    # Standard NumPy
    _, numpy_time = measure_time(np.dot, A, B)
    
    # Convert A and B to JAX arrays
    A_jax = jnp.array(A)
    B_jax = jnp.array(B)

    # JAX NumPy without JIT
    _, jax_time = measure_time(jnp.dot, A_jax, B_jax)

    print(f"Standard NumPy time: {numpy_time:.6f} seconds")
    print(f"JAX NumPy without JIT time: {jax_time:.6f} seconds")

compare_speeds(1000)

Standard NumPy time: 0.017902 seconds
JAX NumPy without JIT time: 0.019902 seconds
